In [8]:
import torch as t
import matplotlib.pyplot as plt
%matplotlib widget
from ipywidgets import FloatSlider, IntSlider, interact, interact_manual
from torch.distributions import Bernoulli
from torch.distributions import MultivariateNormal as MvNormal

<h1> Lecture 3: Bayesian inference for unsupervised learning </h1>
$$
\newcommand{\bracket}[3]{\left#1 #3 \right#2}
\newcommand{\b}{\bracket{(}{)}}
\newcommand{\Bernoulli}{{\rm Bernoulli}\b}
\newcommand{\x}{\mathbf{x}}
\newcommand{\X}{\mathbf{X}}
\newcommand{\Q}{\mathbf{Q}}
\newcommand{\m}{\boldsymbol{\mu}}
\newcommand{\P}{{\rm P}\b}
\newcommand{\dd}[2][]{\frac{\partial #1}{\partial #2}}
\newcommand{\S}{\mathbf{\Sigma}}
\newcommand{\Sh}{\mathbf{\hat{\Sigma}}}
\newcommand{\mh}{\boldsymbol{\hat{\mu}}}
\newcommand{\N}{\mathcal{N}\b}
\newcommand{\det}{\bracket{\lvert}{\rvert}}
\newcommand{\sb}{\bracket{[}{]}}
\newcommand{\E}{\mathbb{E}\sb}
\newcommand{\Var}{{\rm Var}\sb}
\newcommand{\Cov}{{\rm Cov}\sb}
\DeclareMathOperator*{\argmax}{arg\,max}
\DeclareMathOperator*{\argmin}{arg\,min}
\newcommand{\ph}{\hat{p}}
\newcommand{\at}{\bracket{.}{\rvert}}
\newcommand{\w}{\mathbf{w}}
\newcommand{\L}{\mathcal{L}}
\newcommand{\wh}{\mathbf{\hat{w}}}
\newcommand{\y}{\mathbf{y}}
\newcommand{\0}{\mathbf{0}}
\newcommand{\Uniform}{{\rm Uniform}\b}
\newcommand{\Sprior}{\S_\text{prior}}
\newcommand{\Spost}{\S_\text{post}}
\newcommand{\mprior}{\m_\text{prior}}
\newcommand{\mpost}{\m_\text{post}}
$$


<h2>Simple example: A biased coin flip</h2>
$$
\newcommand{\bracket}[3]{\left#1 #3 \right#2}
\newcommand{\b}{\bracket{(}{)}}
\newcommand{\Bernoulli}{{\rm Bernoulli}\b}
$$
The simplest example is a Bernoulli distribution with probability $p$, i.e. a biased coin flip.  We can sample from this distribution using the PyTorch distributions library.

In [9]:

def sample_bernoulli(p):
    return Bernoulli(probs=p).sample((10,))
interact_manual(sample_bernoulli, p=FloatSlider(value=0.5, min=0, max=1, step=0.05));

interactive(children=(FloatSlider(value=0.5, description='p', max=1.0, step=0.05), Button(description='Run Int…

Now, we can also compute the probability of a sample, under this distribution.  Rembmer that $x$ was distributed according to a Bernoulli,

$$P(x| p) = \Bernoulli{x; p}$$

where,

$$\Bernoulli{x; p} = \begin{cases} 1-p &\text{ if } x=0 \\ p &\text{ if } x=1 \end{cases}$$

Thus, we can evaluate the probability of a sample (0. or 1.) under a Bernoulli,

In [10]:
bern = Bernoulli(probs=0.7)
print(bern.log_prob(0.).exp())
print(bern.log_prob(1.).exp())

tensor(0.3000)
tensor(0.7000)


Note: PyTorch only provides the log_prob method, because this is much more numerically stable when doing deep-learning.  We therefore have to exponentiate to get back the probability.

<h3> Maximum likelihood fitting of a Bernoulli </h3>

We begin by specifying the data,

In [11]:
xs = t.tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.])
xs.shape

torch.Size([12])

The likelihood, i.e. the probability of all the data, treated as a function of the parameter, $p$, is

\begin{align}
  \P{\x| p} = \prod_\lambda \P{x_\lambda| p}
\end{align}

But these numbers rapidly shrink (and become too small for standard floating points).  Instead, we work with the log-likelihood,

\begin{align}
  \L(p) = \sum_\lambda \log \P{x_\lambda| p}
\end{align}

Importantly, the log-likelihood is considered a function of the parameter(s), $p$, but not the data, which is considered fixed.

The goal is to find $\ph$ which maximizes $\L(p)$,

\begin{align}
  \ph = \argmax_{p} \L(p)
\end{align}

Lets find $ \ph $!  We can code-up the log-likelihood using,

In [14]:
def log_likelihood(p):
    return Bernoulli(probs=p).log_prob(xs).sum(-1, keepdim=True)

interact(log_likelihood, p=FloatSlider(min=0.01, max=0.99, step=0.01));

interactive(children=(FloatSlider(value=0.01, description='p', max=0.99, min=0.01, step=0.01), Output()), _dom…

Lets plot the log-likelihood,

In [15]:
ps = t.linspace(0.01, 0.99, 99)[:, None]
print(ps.shape)
ps[:6, :]

torch.Size([99, 1])


tensor([[0.0100],
        [0.0200],
        [0.0300],
        [0.0400],
        [0.0500],
        [0.0600]])

In [16]:
lls = log_likelihood(ps)
lls.shape

torch.Size([99, 1])

In [17]:
fig, ax = plt.subplots()
ax.plot(ps, lls)
ax.set_xlabel("p")
ax.set_ylabel("log-likelihood");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In simple cases like this, we could just use the above plot to find the value of $ p $ with the maximal log-likelihood.

But in more complicated cases with many more parameters, this is no longer possible.

Instead, the first approach is to try to find an analytic expression for the $p$ with the maximal log-probability.

Remember,

\begin{align}
  \L(p) &= \sum_\lambda \log \P{x_\lambda| p}\\
  \L(p) &= \b{\sum_\lambda x_\lambda} \log p + \b{N - \sum_\lambda x_\lambda} \log (1-p)
\end{align}

And solve for where the gradient is zero,

\begin{align}
  0 &= \at{\dd[\L(p)]{p}}_{p=\ph}\\
  0 &= \frac{\sum_\lambda x_\lambda}{\ph} - \frac{N - \sum_\lambda x_\lambda}{1-\ph}\\
  \frac{\sum_\lambda x_\lambda}{\ph} &= \frac{N - \sum_\lambda x_\lambda}{1-\ph}\\
  (1-\ph) \sum_\lambda x_\lambda &= \ph \b{N - \sum_\lambda x_\lambda}\\
  \sum_\lambda x_\lambda &= \ph N\\
  \ph &= \tfrac{1}{N} \sum_i x_i
\end{align}

The maximum-likelihood probability is the empirical mean of the data points.  This is sensible, but not at all obvious: we needed to go through the derivation!

Using this derivation, we can write a function that automatically fits a Bernoulli distribution,

In [18]:
def fit_bernoulli(xs):
    p = xs.sum() / xs.shape[-1]
    return Bernoulli(probs=p)

fitted_bernoulli = fit_bernoulli(xs)

such that samples from the fitted Bernoulli look like the data,

In [19]:
xs

tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.])

In [20]:
fitted_bernoulli.sample((12,))

tensor([0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1.])

<h3> Bayesian fitting of a Bernoulli </h3>

Maximum likelihood works well when we have a reasonable number of datapoints.

But what about when we have very little data (e.g. we might have many biased coins, and be able to toss each one a few times).

In particular, lets say we have a population of coins, whose probabilities are drawn from a uniform distribution,

\begin{align}
  \P{p} &= \Uniform{p; 0, 1}
\end{align}

we take out one coin, toss it twice, and get two zeros,

In [21]:
xs = t.tensor([0., 0.])

What can we say about $p$?

Maximum likelihood would tell us that $ \ph = 0 $,

In [22]:
fit_bernoulli(xs)

Bernoulli(probs: 0.0)

Which is a bit strange, because $ p $ could have been quite large, but coincidentally, our two coin-tosses happened to be zeros.

To understand this in a bit more depth, we can run a simulation.

We start by drawing a large number of $ p $'s, from the uniform prior,

In [23]:
N = 10**5
ps = t.rand(N)
fig, ax = plt.subplots()
ax.set_xlabel("p")
ax.set_ylabel("P(p)")
ax.hist(ps, density=True);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Note: the values on the x-axis are probability *densities*, not probabilities.

The probability of being in any given bin is:
\begin{align}
  \int_{p_0}^{p_0 + \delta} dp \; \P{p} \approx \delta \P{p}
\end{align}
i.e. the bin-width times the probability density.

Here, we have $20$ bins, with probability density $1$, and bin width $\delta = 1/20$.

The probability of being in any 1 bin is therefore $\P{p} \times \delta = 1 \times 1/20 = 1/20$, and adding up the $20$ bins gives us $1$.

Then, for each of these $ p $'s, we toss a couple of coins,

In [24]:
xs = Bernoulli(ps).sample((2,))
xs
xs.shape

torch.Size([2, 100000])

Then, we filter out only those values of $ p $ which actually gave us two zeros,

In [25]:
all_zeros = (xs==0.).all(0)
print(all_zeros)
print(all_zeros.sum())
ps_all_zeros = ps[all_zeros]
print(ps_all_zeros.shape)

tensor([False,  True, False,  ..., False,  True, False])
tensor(33303)
torch.Size([33303])


Now, we can plot those $ p $'s

In [26]:
fig, ax = plt.subplots()
ax.set_xlabel("p")
ax.set_ylabel("P(p|x=[0,0])")
ax.hist(ps_all_zeros, density=True);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

This makes sense: we expect the probability to be small, because when we flipped the coin, we observed two zeros.

But the probability could still be large, with the two zeros we observed being coincidences.

It turns out this idea (sampling until we get something very close to the data, then looking at the corresponding latents, here the probability, $p$), is a real algorithm, called "approximate Bayesian computation" (ABC).

But its a terrible idea: don't do it in practice unless you really have to.  For any non-trivial dataset, you will be waiting a very, very long time before random sampling produces something "sufficient close".

Instead, can we do exact Bayesian inference to directly compute the distribution $\P{p| \x}$.

The answer is yes!

In particular, the law of joint probability tells us that we can write the joint, $\P{\x, p}$ in two equivalent forms,

\begin{align}
  \P{\x, p} &= \P{\x| p} \P{p} = \P{p| \x} \P{\x}
\end{align}

The first form $\P{\x| p} \P{p}$, is the standard one, and we can readily compute it given the expressions above.  The second form, $\P{p| \x} \P{\x}$ is a bit more unusual: it isn't immediately obvious how we can compute these terms.

Nonetheless, we can rearrange to compute the term we're interested in,

\begin{align}
  \P{p| \x} &= \frac{\P{\x| p} \P{p}}{\P{\x}} \propto \P{\x| p} \P{p}\\
  \log \P{p| \x} &= \L\b{p} + \log \P{p} + \text{const}
\end{align}

The proportionality arises because we take data, $\x$, to be fixed, so we only care about parameter, $p$ dependence (as long as we make sure the distribution normalizes!

Lets do it!

In [27]:
xs = t.Tensor([0., 0.])

def log_prior(p):
    return 0.

def log_likelihood(p):
    return Bernoulli(p).log_prob(xs).sum(-1, keepdim=True)

ps = t.linspace(0.005, 0.995, 100).view(-1, 1)
dp = ps[1] - ps[0]
    
unnorm_log_posterior = log_prior(ps) + log_likelihood(ps)
unnorm_posterior = unnorm_log_posterior.exp()
norm_posterior = unnorm_posterior / (dp * unnorm_posterior.sum())

fig, ax = plt.subplots()
ax.set_xlabel("p")
ax.set_ylabel("P(p|x=[0,0])")
ax.hist(ps_all_zeros, density=True);
ax.plot(ps, norm_posterior);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

It is possible to analytically compute the distribution over $p$ for a coin-toss.  But this doesn't give us much additional insight.  Instead, we'll do this below for a Gaussian distribution.

<h2>More interesting examples</h2>
In the previous example, a datapoint was either 0 or 1,

```
X = {0, 1}
```

but $ x $ could take on almost any other type.  Common examples include:

```
X = Vector{Float}   # A vector
X = Str             # A string
X = Image           # An image
```

Complex, state-of-the-art models for images (GANs) and text (GTP-2) do exactly what we described above.
Then, they take a large dataset of images/text do a very large amount of processing, to fit a distribution to that data.
Once the distribution has been fitted, they can draw samples from that distribution, that should look like the data.

<h2> Multivariate Normal (Gaussian)</h2>
The multivariate Normal is the most important distribution over vectors.

\begin{align}
  \P{\x| \m, \S} &= \N{\x; \m, \S}\\
  \log \N{\x; \m, \S} &= -\tfrac{1}{2} \log \det{2 \pi \S} - \tfrac{1}{2} \b{\x - \m} \S^{-1} \b{\x-\m}^T
\end{align}

The expectation of the distribution is given by $\m$ and the covariance is given by $\S$,

\begin{align}
  \E{\x} &= \m\\
  \Cov{\x} &= \S
\end{align}

In code,

In [28]:
from torch.distributions import MultivariateNormal as MvNormal
mu = t.ones(2)
Sigma = t.ones(2,2) + t.eye(2)
dist = MvNormal(mu, Sigma)

In [29]:
N = 10**6
X = dist.sample((N,))
print(X[:6, :])
print(X.shape)
mu.shape

tensor([[ 1.0071,  1.5192],
        [ 0.3646,  1.7885],
        [-0.0255,  2.4669],
        [-0.7367,  2.1331],
        [ 2.0999,  0.5540],
        [ 2.7106,  1.2354]])
torch.Size([1000000, 2])


torch.Size([2])

<h3> The mean of the multivariate normal </h3>
The expected value of $\x$ is given by,

\begin{align}
  \m  &= \E{\x}\\
\end{align}

so the empirical mean is,

\begin{align}
  \mh &= \tfrac{1}{N} \sum_{\lambda=1}^N \x_{\lambda}
\end{align}

and as $ \x_\lambda = X_{\lambda, :} $

\begin{align}
  \hat{\mu}_i &= \tfrac{1}{N} \sum_{\lambda=1}^N X_{\lambda i}
\end{align}

Now we can check that the true mean is close to the empirical mean,

In [30]:
print(mu)
mh = X.sum(0)/N
print(mh)

tensor([1., 1.])
tensor([1.0025, 1.0010])


<h3> The covariance of the multivariate normal</h3>
While the mean characterises the location of the data, the covariance characterises its shape.

The covariance is defined as,

\begin{align}
  \S &= \E{\b{\x - \E{\x}} \b{\x - \E{\x}}^T}
\end{align}

so the empirical covariance is,

\begin{align}
  \Sh &= \tfrac{1}{N} \sum_{\lambda=1}^N \b{\x_\lambda - \mh} \b{\x_\lambda - \mh}^T\\
  \hat{\Sigma}_{ij} &= \tfrac{1}{N} \sum_{\lambda=1}^N \b{X_{\lambda i} - \hat{\mu}_{i}} \b{X_{\lambda j} - \hat{\mu}_{j}}\\
  \Sh &= \tfrac{1}{N} \b{\X - \mh}^T \b{\X - \mh}
\end{align}

Lets check this expression,

In [31]:
print(Sigma)
Sh = (X-mh).T @ (X-mh) / N
print(Sh)

tensor([[2., 1.],
        [1., 2.]])
tensor([[2.0025, 1.0018],
        [1.0018, 1.9992]])


To get some intuition for what the covariance means, we can plot samples from a multivariate normal with the same variance for $x_0$ and $x_1$, but different covariances,

In [32]:
def plot(c):
    fig, ax = plt.subplots()
    ax.set_xlabel("$x_0$")
    ax.set_ylabel("$x_1$")
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    
    mu = t.zeros(2)
    Sigma = c*t.ones(2, 2) + (1-c)*t.eye(2)
    print("Sigma =")
    print(Sigma)
    dist = MvNormal(mu, Sigma)
    xs = dist.sample((10000,))
    ax.scatter(xs[:, 0], xs[:, 1])
    
interact_manual(plot, c=FloatSlider(value=0, min=-0.99, max=0.99, step=0.01));

interactive(children=(FloatSlider(value=0.0, description='c', max=0.99, min=-0.99, step=0.01), Button(descript…

To begin, consider positive $\Cov{x_0, x_1} = \E{x_0 x_1}$.  To get positive covariances, $x_0$ and $x_1$ will tend to have the same sign (either both positive or both negative), so the overall product, $x_0 x_1$ is usually positive.

Now, consider negative $\Cov{x_0, x_1} = \E{x_0 x_1}$.  To get negative covariances, $x_0$ and $x_1$ will tend to have the different signs (one positive and the other negative), so the overall product, $x_0 x_1$ is usually negative.

Now, consider $0 = \Cov{x_0, x_1} = \E{x_0 x_1}$.  In this case, $x_0$ and $x_1$ are unrelated.

As the covariance approaches the variance, the distribution gets narrower.

Until eventually, the only way of achieving,

\begin{align}
  1 = \Var{x_0} = \Var{x_1} \approx \Cov{x_0, x_1} = \E{x_0 x_1}
\end{align}

is by setting,

\begin{align}
  x_0 = x_1
\end{align}

<h3>Maximum likelihood fitting</h3>

Maximum likelihood parameters, $\m$ and $\S$, for a multivariate normal is given by the empirical mean, $\mh$ and covariance $\Sh$,

\begin{align}
  \mh, \Sh = \argmax_{\m, \S} \sb{\sum_\lambda \log \N{\x_\lambda| \m, \S}}
\end{align}

This seems sensible, but it is not at all obvious!  (Left as an exercise for the reader.)

<h3> Bayesian fitting of $\mu$ </h3>

We assume that $\m$ has a multivariate Normal prior,

\begin{align}
  \P{\m} &= \N{\m; \0, \Sprior}\\
  \P{\x_\lambda| \m} &= \N{\x_\lambda; \m, \Q}
\end{align}

In [33]:
Sprior = 2*t.eye(2)
Q = t.tensor([[0.3, 0.2], [0.2, 0.5]])
Pm = MvNormal(t.zeros(2), Sprior)
def sample_X(N):
    m = Pm.sample()
    X = MvNormal(m, Q).sample((N,))
    
    fig, ax = plt.subplots()
    ax.set_xlabel("$x_0$")
    ax.set_ylabel("$x_1$")
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.scatter(X[:, 0], X[:, 1])
    ax.scatter(m[0:1], m[1:2])
    
interact_manual(sample_X, N=IntSlider(min=1, max=100, value=100));

interactive(children=(IntSlider(value=100, description='N', min=1), Button(description='Run Interact', style=B…

And our goal is to compute,

\begin{align}
  \P{\m| \X} &\propto \P{\m} \prod_\lambda \P{\x_\lambda| \m}\\
  \log \P{\m| \X} &= \log \P{\m} + \sum_\lambda \log \P{\x_\lambda| \m} + \text{const}
\end{align}

First, we write out the log-probability under the prior,

\begin{align}
  \log \P{\m} &= -\underbrace{\tfrac{1}{2} \m^T \S^{-1} \m}_\text{quadratic} + \text{const}
\end{align}

Second, we write out the log-likelihood for a single data-point,

\begin{align}
  \log \P{\x_\lambda| \m} &= -\tfrac{1}{2} \b{\x_\lambda - \m}^T \Q^{-1} \b{\x_\lambda - \m} + \text{const}\\
  \log \P{\x_\lambda| \m} &= -\tfrac{1}{2} \sb{\m^T \Q^{-1} \m - \x_\lambda^T \Q^{-1} \m - \m^T \Q^{-1} \x_\lambda + \x_\lambda^T \Q^{-1} \x_\lambda} + \text{const}
\end{align}

As,

\begin{align}
  \mathbf{a}^T \mathbf{b} = \mathbf{b}^T \mathbf{a} = \sum_i a_i b_i,
\end{align}

we have,

\begin{align}
  \m^T \Q^{-1} \x_\lambda = \m^T \b{\Q^{-1}}^T \x_\lambda = \b{\Q^{-1} \m}^T \x_\lambda = \x_\lambda^T \b{\Q^{-1} \m} = \x_\lambda^T \Q^{-1} \m
\end{align}

And as $\x_\lambda^T \Q^{-1} \x_\lambda$ does not vary with $\m$, we can incorporate that term into the constant,

\begin{align}
  \log \P{\x_\lambda| \m} &=\underbrace{-\tfrac{1}{2} \m^T \Q^{-1} \m}_\text{quadratic} + \underbrace{\m^T \Q^{-1} \x_\lambda}_\text{linear} + \text{const}.
\end{align}



The log-prior and log-likelihood are quadratic functions of $\m$, so we know the posterior must also be Gaussian.

The easiest way of identifying the posterior mean and covariance is to name them $\mpost$ and $\Spost$, and compute the corresponding quadratic and linear terms,

\begin{align}
  \log \P{\m| \X} &= -\tfrac{1}{2} \b{\m - \mpost}^T \Spost^{-1} \b{\m - \mpost} + \text{const}\\
  &= -\tfrac{1}{2} \sb{\m^T \Spost^{-1} \m - \mpost^T \Spost^{-1} \m - \m^T \Spost^{-1} \mpost + \mpost^T \Spost^{-1} \mpost} + \text{const}\\
    &=\underbrace{-\tfrac{1}{2} \m^T \Spost^{-1} \m}_\text{quadratic} + \underbrace{\m^T \Spost^{-1} \mpost}_\text{linear} + \text{const}.
\end{align}

Now, we know,

\begin{align}
  \log \P{\m| \X} &= \log \P{\m} + \sum_\lambda \log \P{\x_\lambda| \m} + \text{const}\\
  -\underbrace{\tfrac{1}{2} \m^T \Spost^{-1} \m}_\text{quadratic} + \underbrace{\m^T \Spost^{-1} \mpost}_\text{linear} &= - \underbrace{\tfrac{1}{2} \m^T \Sprior^{-1} \m}_\text{quadratic} + \sum_\lambda \sb{\underbrace{-\tfrac{1}{2} \m^T \Q^{-1} \m}_\text{quadratic} + \underbrace{\m^T \Q^{-1} \x_\lambda}_\text{linear}}
\end{align}

As this must be true for all settings of $\m$, we can separately match the liner and quadratic terms.

For the quadratic terms,
\begin{align}
  -\tfrac{1}{2} \m^T \Spost^{-1} \m &= - \tfrac{1}{2} \m^T \Sprior^{-1} \m - \sum_\lambda \sb{\tfrac{1}{2} \m^T \Q^{-1} \m}
\end{align}

As this must be true for all $\m$,

\begin{align}
  \Spost^{-1} &= \Sprior^{-1} + \sum_\lambda \Q^{-1}\\
  \Spost^{-1} &= \Sprior^{-1} + N \Q^{-1}\\
  \Spost &= \b{\Sprior^{-1} + N \Q^{-1}}^{-1}
\end{align}

And for the linear terms,

\begin{align}
  \m^T \Spost^{-1} \mpost &= \sum_\lambda \m^T \Q^{-1} \x_\lambda\\
  \m^T \Spost^{-1} \mpost &= N \m^T \Q^{-1} \langle\x\rangle\\
\end{align}

where,

\begin{align}
  \langle \x \rangle = \tfrac{1}{N} \sum_{\lambda=1}^N \x_\lambda.
\end{align}

As this must be true for all $\m$,

\begin{align}
  \Spost^{-1} \mpost &= N \Q^{-1} \langle\x\rangle\\
  \mpost &= N \Spost \Q^{-1} \langle\x\rangle\\
\end{align}

As a sanity-check, consider a very large number of data points, $N$,

In that case, 
\begin{align}
  \Sprior^{-1} + N \Q^{-1} \approx N \Q^{-1}
  \Spost = \b{\Sprior^{-1} + N \Q^{-1}}^{-1} \approx \b{N \Q^{-1}}^{-1} = \tfrac{1}{N} \Q
\end{align}

so,

\begin{align}
  \mpost &\approx N \b{\tfrac{1}{N} \Q} \Q^{-1} \langle\x\rangle = \langle \x \rangle
\end{align}
